In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [2]:
source_dir = '../data/undesired_outcomes_pred/'
dest_dir = source_dir
extension = '.csv'

In [3]:
elog_uo = pd.read_csv(source_dir.__add__('elog_uo.csv'))

In [4]:
pd.options.display.max_columns = 100

In [5]:
elog_uo.sample(10)

,Case ID,Activity,Resource,Complete Timestamp,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,(case) applicant,(case) area,(case) basic payment,(case) cross_compliance,(case) department,(case) greening,(case) number_parcels,(case) payment_actual0,(case) payment_actual1,(case) payment_actual2,(case) payment_actual3,(case) penalty_ABP,(case) penalty_AGP,(case) penalty_AJLP,(case) penalty_AUVP,(case) penalty_AVBP,(case) penalty_AVGP,(case) penalty_AVJLP,(case) penalty_AVUVP,(case) penalty_B16,(case) penalty_B2,(case) penalty_B3,(case) penalty_B4,(case) penalty_B5,(case) penalty_B5F,(case) penalty_B6,(case) penalty_BGK,(case) penalty_BGKV,(case) penalty_BGP,(case) penalty_C16,(case) penalty_C4,(case) penalty_C9,(case) penalty_CC,(case) penalty_GP1,(case) penalty_JLP1,(case) penalty_JLP2,(case) penalty_JLP3,(case) penalty_JLP5,(case) penalty_JLP6,(case) penalty_JLP7,(case) penalty_V5,(case) penalty_amount0,(case) penalty_amount1,(case) penalty_amount2,(case) penalty_amount3,(case) redistribution,(case) rejected,(case) risk_factor,(case) selected_manually,(case) selected_random,(case) selected_risk,(case) small farmer,(case) year,(case) young farmer,activity,docid,doctype,eventid,note,subprocess,success,num changes applied amount,final applied amount,diff applied amount,applied amount lowered,applied amount changed,num changes actual amount,final actual amount,diff actual amount,actual amount lowered,actual amount changed,actual amount pay,num changes penalty amount,final penalty amount,penalty amount applied,num_penalty_reasons,num_severe_penalties,penalty applied,severe penalty applied,timestamp_year,timestamp_month,undesired_outcome1,undesired_outcome2,undesired_outcome
136628,2cd9f6ec6f813adf,Payment application-Application-finish editing,75992a,2017-11-21 07:56:11.016,1653.47,NaN,NaN,NaN,6f8022fe20c7a814,4.9630,True,0.0,6b,True,4,1653.47,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.00,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2017,False,finish editing,ff40057d15bd77ee,Payment application,ff00057f6be77900,NaN,Application,True,0,1653.47,0.0,False,False,0,1653.47,0.0,False,False,False,0,0.00,False,0,0,False,False,2017,11,True,False,True
92885,18b91764e05a84de,Payment application-Application-finish editing,727350,2016-12-15 13:11:49.799,9679.96,NaN,NaN,NaN,4b755d78ba475789,28.4699,True,0.0,6b,True,17,9670.06,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,13.02,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2016,False,finish editing,ff00055ed565c503,Payment application,ff400563f52ae028,automatic,Application,True,0,9679.96,0.0,False,False,0,9670.06,0.0,False,False,False,0,13.02,True,1,0,True,False,2016,12,False,False,False
83569,165a14a7779fb04c,Payment application-Application-finish editing,727350,2016-12-15 09:18:07.183,15177.32,NaN,NaN,NaN,c462c5c400dc82e8,47.4960,True,0.0,e7,True,31,15182.90,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.00,NaN,NaN,NaN,True,False,1.0,False,False,False,False,2016,False,finish editing,ff00055ed577cedf,Payment application,ff400563f4541620,automatic,Application,True,0,15177.32,0.0,False,False,0,15182.90,0.0,False,False,False,0,0.00,False,0,0,False,False,2016,12,True,False,True
119896,786e54de21b79454,Payment application-Application-finish editing,727350,2017-12-07 17:10:54.423,1866.70,NaN,NaN,NaN,cc4c99b03c99b8d5,5.6696,True,0.0,e7,True,4,1283.61,NaN,NaN,NaN,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [6]:
target = np.array(elog_uo['undesired_outcome'])

In [7]:
elog_uo.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp',
       '(case) amount_applied0', '(case) amount_applied1',
       '(case) amount_applied2', '(case) amount_applied3', '(case) applicant',
       '(case) area', '(case) basic payment', '(case) cross_compliance',
       '(case) department', '(case) greening', '(case) number_parcels',
       '(case) payment_actual0', '(case) payment_actual1',
       '(case) payment_actual2', '(case) payment_actual3',
       '(case) penalty_ABP', '(case) penalty_AGP', '(case) penalty_AJLP',
       '(case) penalty_AUVP', '(case) penalty_AVBP', '(case) penalty_AVGP',
       '(case) penalty_AVJLP', '(case) penalty_AVUVP', '(case) penalty_B16',
       '(case) penalty_B2', '(case) penalty_B3', '(case) penalty_B4',
       '(case) penalty_B5', '(case) penalty_B5F', '(case) penalty_B6',
       '(case) penalty_BGK', '(case) penalty_BGKV', '(case) penalty_BGP',
       '(case) penalty_C16', '(case) penalty_C4', '(case) penalty_C9',
       '(case) penalty

In [9]:
features = ['(case) amount_applied0', '(case) area', '(case) department', '(case) number_parcels', '(case) payment_actual0',
           '(case) penalty_amount0', '(case) penalty_B2', '(case) year', 'final applied amount',
             'final actual amount', 'final penalty amount', 'num_penalty_reasons', 'num_severe_penalties',
            'num changes actual amount', 'num changes applied amount', 'num changes penalty amount', 'penalty amount applied', 
            'penalty applied', 'severe penalty applied', 'timestamp_month', 'timestamp_year']

In [10]:
elog_features = elog_uo[features]

In [12]:
elog_features.shape[1] == len(features)

True

In [23]:
elog_features = pd.get_dummies(elog_features) # one hot encoding

In [24]:
elog_features_np = np.array(elog_features)
elog_features_col = list(elog_features.columns)

In [25]:
train_x, test_x, train_y, test_y = train_test_split(elog_features_np, target, test_size = 0.3, random_state = 99)

In [26]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 99)

In [27]:
rf.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=99,
           verbose=0, warm_start=False)

In [52]:
pred = rf.predict(test_x)
pred = pred.astype(int)

In [53]:
errors = abs(pred - test_y)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.03 degrees.


In [54]:
mape = 100 * (errors / test_y)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [56]:
accuracy = accuracy_score(test_y, pred)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 0.97 %.


In [57]:
confusion_matrix(test_y, pred)

array([[20818,     3],
       [ 1341, 19339]])